In [ ]:
### 마곡역 근처 top5 아파트 시세
# 마곡엠벨리 14단지, 마곡엠벨리 12단지, 마곡엠벨리 15단지, 마곡엠벨리 9단지, 마곡힐스테이트

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [3]:
## 복호화함수
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad



def decrypt(encrypted_text, secret):
    """
    AES ECB 모드로 암호화된 텍스트를 복호화합니다.
    
    Args:
        encrypted_text (str): Base64로 인코딩된 암호화된 텍스트
    
    Returns:
        str: 복호화된 평문 텍스트
    """
    # Base64 디코딩
    encrypted_bytes = base64.b64decode(encrypted_text)
    
    # 키 생성
    key = get_key(secret)
    
    # AES 복호화 객체 생성 (ECB 모드)
    cipher = AES.new(key, AES.MODE_ECB)
    
    # 복호화
    decrypted_bytes = cipher.decrypt(encrypted_bytes)
    
    # PKCS7 패딩 제거
    unpadded_bytes = unpad(decrypted_bytes, AES.block_size)
    
    # UTF-8 문자열로 변환
    return unpadded_bytes.decode('utf-8')

def get_key(secret):
    """
    비밀 문자열을 AES 키로 변환합니다.
    키 길이에 따라 적절한 크기로 조정합니다.
    
    Args:
        secret (str): 비밀 문자열
    
    Returns:
        bytes: AES 키 (16, 24, 또는 32 바이트)
    """
    # 문자열을 UTF-8 바이트로 변환
    key_bytes = secret.encode('utf-8')
    length = len(key_bytes)
    
    # 키 길이에 따라 조정
    if length < 16:
        # 16바이트보다 작으면 16바이트로 패딩
        adjusted_key = key_bytes[:16].ljust(16, b'\x00')
    elif length > 16 and length < 24:
        # 16바이트보다 크고 24바이트보다 작으면 24바이트로 조정
        adjusted_key = key_bytes[:24].ljust(24, b'\x00')
    elif length > 24 and length < 32:
        # 24바이트보다 크고 32바이트보다 작으면 32바이트로 조정
        adjusted_key = key_bytes[:32].ljust(32, b'\x00')
    elif length >= 32:
        # 32바이트 이상이면 32바이트로 잘라내기
        adjusted_key = key_bytes[:32]
    else:
        # 정확히 16바이트면 그대로 사용
        adjusted_key = key_bytes
    
    return adjusted_key

In [4]:
# 억단위 숫자로
import re

def parse_korean_price(text):
    """
    '1억 2,300' → 123000000 변환
    """
    if pd.isna(text):
        return None
    text = text.replace(',', '').replace(' ', '')
    total = 0
    match = re.match(r'(?:(\d+)억)?(?:(\d+)천)?(\d+)?', text)
    if not match:
        return None
    if match.group(1):  # 억
        total += int(match.group(1)) * 100_000_000
    if match.group(2):  # 천만 단위
        total += int(match.group(2)) * 10_000_000
    if match.group(3):  # 만 단위 이하
        total += int(match.group(3))
    return total

In [5]:
## 마곡역 마곡엠벨리 14단지
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20288954"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75436879251857920288954']


In [8]:
## 마곡 엠벨리 14단지
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20288954&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_1 = pd.DataFrame(rows)

key = '75436879251857920288954'

# 가격과 전월세 컬럼 복호화
df_1['가격'] = df_1['가격'].apply(lambda x: decrypt(x, key))
df_1['전월세'] = df_1['전월세'].apply(lambda x: decrypt(x, key))

df_1['가격(숫자)'] = df_1['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_1

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.24,15,16억,,1600000000
1,2025.06.13,15,"15억 5,800",,1500005800
2,2025.05.31,3,"14억 1,500",,1400001500
3,2025.05.03,12,"15억 1,000",,1500001000
4,2025.03.29,6,"17억 8,000",,1700008000
...,...,...,...,...,...
166,2014.07.17,2,5억 650,,500000650
167,2014.07.16,3,"4억 4,740",,400004740
168,2014.07.15,7,"4억 2,650",,400002650
169,2014.07.15,9,"4억 7,441",,400007441


In [7]:
## 마곡역 마곡엠벨리 12단지
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20342347"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75736879054303820342347']


In [13]:
## 마곡 엠벨리 12단지
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20342347&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_2 = pd.DataFrame(rows)

key = '75736879054303820342347'

# 가격과 전월세 컬럼 복호화
df_2['가격'] = df_2['가격'].apply(lambda x: decrypt(x, key))
df_2['전월세'] = df_2['전월세'].apply(lambda x: decrypt(x, key))


df_2['가격(숫자)'] = df_2['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수

df_2

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.26,9,"14억 9,000",,1400009000
1,2025.05.23,7,"14억 3,500",,1400003500
2,2025.04.25,6,"12억 4,500",,1200004500
3,2024.07.24,10,12억,,1200000000
4,2024.07.08,6,"13억 5,500",,1300005500
5,2024.06.08,12,"11억 5,000",,1100005000
6,2024.05.15,12,"13억 5,950",,1300005950
7,2024.05.11,13,"11억 2,300",,1100002300
8,2024.05.07,15,"10억 9,500",,1000009500
9,2024.04.06,9,"12억 8,000",,1200008000


In [14]:
## 마곡역 마곡엠벨리 15단지
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20288938"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75836879251917120288938']


In [15]:
## 마곡 엠벨리 15단지
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20288938&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_3 = pd.DataFrame(rows)

key = '75836879251917120288938'

# 가격과 전월세 컬럼 복호화
df_3['가격'] = df_3['가격'].apply(lambda x: decrypt(x, key))
df_3['전월세'] = df_3['전월세'].apply(lambda x: decrypt(x, key))


df_3['가격(숫자)'] = df_3['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수

df_3

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.27,3,"14억 1,000",,1400001000
1,2025.06.19,5,"14억 7,800",,1400007800
2,2025.06.13,14,"14억 9,000",,1400009000
3,2025.05.14,3,"14억 2,000",,1400002000
4,2025.04.19,10,"15억 1,000",,1500001000
...,...,...,...,...,...
168,2014.07.15,13,"3억 4,800",,300004800
169,2014.07.14,8,"4억 3,894",,400003894
170,2014.06.25,15,"3억 5,191",,300005191
171,2014.06.20,14,"4억 3,500",,400003500


In [16]:
## 마곡역 마곡엠벨리 9단지
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=1500070603"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7533140405975661500070603']


In [17]:
## 마곡 엠벨리 9단지
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=1500070603&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_4 = pd.DataFrame(rows)

key = '7533140405975661500070603'

# 가격과 전월세 컬럼 복호화
df_4['가격'] = df_4['가격'].apply(lambda x: decrypt(x, key))
df_4['전월세'] = df_4['전월세'].apply(lambda x: decrypt(x, key))


df_4['가격(숫자)'] = df_4['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수

df_4

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.16,13,"14억 3,000",,1400003000
1,2025.07.11,8,"12억 5,500",,1200005500
2,2025.06.28,4,"14억 9,000",,1400009000
3,2025.06.27,2,12억,,1200000000
4,2025.06.27,11,"12억 6,000",,1200006000
...,...,...,...,...,...
73,2020.07.31,15,9억 384,,900000384
74,2020.06.29,14,"6억 5,593",,600005593
75,2020.06.29,12,7억 418,,700000418
76,2020.06.26,5,"8억 2,716",,800002716


In [18]:
## 마곡역 마곡힐스테이트
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20330992"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75236879096317320330992']


In [19]:
## 마곡 힐스테이트
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20330992&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_5 = pd.DataFrame(rows)

key = '75236879096317320330992'

# 가격과 전월세 컬럼 복호화
df_5['가격'] = df_5['가격'].apply(lambda x: decrypt(x, key))
df_5['전월세'] = df_5['전월세'].apply(lambda x: decrypt(x, key))


df_5['가격(숫자)'] = df_5['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수

df_5

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.28,7,"14억 5,000",,1400005000
1,2025.06.24,7,"8억 9,500",,800009500
2,2025.06.07,7,"14억 7,000",,1400007000
3,2025.04.16,11,"14억 3,000",,1400003000
4,2025.04.16,11,"14억 3,000",,1400003000
...,...,...,...,...,...
164,2017.09.11,15,"10억 8,000",,1000008000
165,2017.08.16,4,"6억 8,000",,600008000
166,2017.08.04,13,7억,,700000000
167,2017.07.26,7,"7억 3,500",,700003500
